<a href="https://colab.research.google.com/github/Srinivas2109/testbot/blob/main/gemma_connection_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install git+https://github.com/huggingface/transformers -U
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install trl
!pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-cjbvqoks
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-cjbvqoks
  Resolved https://github.com/huggingface/transformers to commit 6a05f68f51ef620eb85b6f4f2681bf5bacf15ae7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.43.0.dev0-py3-none-any.whl size=9340557 sha256=2c87c89f4960dcc3c23c567221868f07a4f80b92be2b7f65313ead5c52d54856
  Stored in directory: /tmp/pip-ephem-wheel-cache-p2hxc1xe/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
notebook_login(write_permission=True)

In [ ]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Define conversation template
chat_template = """
**You:** {user_input}
**Gemma-7b:** {model_response}
"""

# Load pre-trained model and tokenizer
#model_id = "google/gemma-2b-it"
#lora_config = BitsAndBytesConfig(
#    load_in_4bit=True,
 #   bnb_4bit_quant_type="nf4",
  #  bnb_4bit_compute_dtype=torch.bfloat16
#)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", quantization_config=quantization_config)



# tokenizer = AutoTokenizer.from_pretrained(model_id)
# quantized_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=lora_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def chatbot():
  """
  Interactive chat function with Gemma-2b chatbot.
  """
  while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break

    # Encode user input with tokenizer
   # input_ids = tokenizer.encode(user_input, return_tensors="pt")

    # Generate model response

    input_ids = tokenizer(user_input, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    # output = quantized_model.generate(input_ids, max_length=1024, do_sample=True, top_k=10, top_p=0.9)
    # model_response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print conversation

if __name__ == "__main__":
    print("Welcome to the Gemma-7b Chatbot!")
    chatbot()
    print("Goodbye!")

Welcome to the Gemma-7b Chatbot!
hello, i am trying to find a way to convert a list of words into a list of
super ki nuvvu?

I am a language model, and I am here to help
